In [ ]:
import sys
import os
import csv
import cv2
import numpy as np
from datetime import datetime
from PIL import Image
from PyQt5.QtWidgets import (
    QApplication, QWidget, QFileDialog, QLabel, QPushButton,
    QListWidget, QProgressBar, QTableWidget, QTableWidgetItem,
    QGroupBox, QVBoxLayout, QHBoxLayout, QSizePolicy
)
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt

def cv_imread_unicode(path):
    try:
        img = Image.open(path)
        return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    except Exception as e:
        print(f"이미지 열기 실패: {e}")
        return None

class DetectionUI(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Target Detection UI")
        self.setGeometry(100, 100, 1200, 700)

        self.target_images = []
        self.cycle_images = []
        self.current_index = 0
        self.sim_threshold = 0.8

        main_layout = QHBoxLayout(self)

        # Left panel
        left_box = QVBoxLayout()
        tg_group = QGroupBox("Select 4 Target Images")
        tg_layout = QVBoxLayout()
        self.tg_list = QListWidget()
        btn_tg = QPushButton("Load Target Images")
        btn_tg.clicked.connect(self.load_targets)
        tg_layout.addWidget(self.tg_list)
        tg_layout.addWidget(btn_tg)
        tg_group.setLayout(tg_layout)

        cyc_group = QGroupBox("Select Cycle Images (up to 100)")
        cyc_layout = QVBoxLayout()
        self.cyc_list = QListWidget()
        btn_cyc = QPushButton("Load Cycle Images")
        btn_cyc.clicked.connect(self.load_cycle)
        cyc_layout.addWidget(self.cyc_list)
        cyc_layout.addWidget(btn_cyc)
        cyc_group.setLayout(cyc_layout)

        left_box.addWidget(tg_group)
        left_box.addWidget(cyc_group)
        main_layout.addLayout(left_box, 1)

        # Center panel
        center_box = QVBoxLayout()
        img_nav = QHBoxLayout()
        self.lbl_prev = QPushButton("\u25c0")
        self.lbl_prev.clicked.connect(self.show_prev)
        self.lbl_prev.setEnabled(False)
        self.lbl_image = QLabel("No Image")
        self.lbl_image.setAlignment(Qt.AlignCenter)
        self.lbl_image.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        self.lbl_next = QPushButton("\u25b6")
        self.lbl_next.clicked.connect(self.show_next)
        self.lbl_next.setEnabled(False)
        img_nav.addWidget(self.lbl_prev)
        img_nav.addWidget(self.lbl_image, 1)
        img_nav.addWidget(self.lbl_next)

        self.lbl_sim = QLabel("Similarity: N/A")
        self.lbl_sim.setAlignment(Qt.AlignCenter)
        self.progress = QProgressBar()
        self.progress.setValue(0)

        btn_save = QPushButton("Save to CSV")
        btn_save.clicked.connect(self.save_to_csv)

        center_box.addLayout(img_nav, 5)
        center_box.addWidget(self.lbl_sim)
        center_box.addWidget(self.progress)
        center_box.addWidget(btn_save)
        main_layout.addLayout(center_box, 2)

        # Right panel
        right_box = QVBoxLayout()
        record_header = QHBoxLayout()
        lbl_records = QLabel("Detection Records")
        lbl_records.setAlignment(Qt.AlignCenter)
        btn_reset = QPushButton("\ucd08\uae30\ud654")
        btn_reset.clicked.connect(self.reset_results)
        record_header.addWidget(lbl_records)
        record_header.addStretch()
        record_header.addWidget(btn_reset)

        self.table = QTableWidget(0, 3)
        self.table.setHorizontalHeaderLabels(["Time", "Image Name", "Result"])
        self.table.horizontalHeader().setStretchLastSection(True)

        right_box.addLayout(record_header)
        right_box.addWidget(self.table)
        main_layout.addLayout(right_box, 2)

    def load_targets(self):
        files, _ = QFileDialog.getOpenFileNames(
            self, "Select 4 Target Images", "", "Images (*.png *.jpg *.bmp)"
        )
        if files:
            self.target_images = files[:4]
            self.tg_list.clear()
            for f in self.target_images:
                self.tg_list.addItem(os.path.basename(f))

    def load_cycle(self):
        files, _ = QFileDialog.getOpenFileNames(
            self, "Select Cycle Images", "", "Images (*.png *.jpg *.bmp)"
        )
        if files:
            self.cycle_images = files[:100]
            self.cyc_list.clear()
            for f in self.cycle_images:
                self.cyc_list.addItem(os.path.basename(f))
            self.current_index = 0
            self.update_display_controls()
            self.show_image()  # 즉시 유사도 계산되도록 추가

    def update_display_controls(self):
        has = len(self.cycle_images) > 0
        self.lbl_prev.setEnabled(has)
        self.lbl_next.setEnabled(has)

    def full_preprocessing(self, img):
        if img is None:
            return None
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(gray)
        binary = cv2.adaptiveThreshold(enhanced, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY_INV, 11, 2)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        morph = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        filtered = cv2.bilateralFilter(morph, 9, 75, 75)
        return filtered

    def calculate_similarity(self, current_img_path):
        if not self.target_images:
            print("❌ 타겟 이미지가 없습니다!")
            return None

        img2 = cv_imread_unicode(current_img_path)
        if img2 is None:
            return None
        img2 = self.full_preprocessing(img2)
        if img2 is None:
            return None

        orb = cv2.ORB_create()
        max_score = 0
        for path in self.target_images:
            img1 = cv_imread_unicode(path)
            if img1 is None:
                continue
            img1 = self.full_preprocessing(img1)
            if img1 is None:
                continue
            kp1, des1 = orb.detectAndCompute(img1, None)
            kp2, des2 = orb.detectAndCompute(img2, None)
            if des1 is not None and des2 is not None:
                bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                matches = bf.match(des1, des2)
                score = len(matches)
                if score > max_score:
                    max_score = score
        return min(max_score / 100, 1.0)

    def show_image(self):
        if not self.cycle_images:
            return
        path = self.cycle_images[self.current_index]
        pix = QPixmap(path).scaled(400, 300, Qt.KeepAspectRatio)
        self.lbl_image.setPixmap(pix)

        sim_val = self.calculate_similarity(path)
        print(f"🔍 현재 이미지: {os.path.basename(path)}, 유사도: {sim_val}")
        if sim_val is None:
            result = "Skipped (Invalid)"
            sim_display = "N/A"
        else:
            result = "Detected" if sim_val > self.sim_threshold else "Not Detected"
            sim_display = f"{sim_val:.2f}"

        self.lbl_sim.setText(f"Similarity: {sim_display} ({result})")
        total = len(self.cycle_images)
        self.progress.setValue(int((self.current_index + 1) / total * 100))

        now = datetime.now().strftime("%H:%M:%S")
        row = self.table.rowCount()
        self.table.insertRow(row)
        self.table.setItem(row, 0, QTableWidgetItem(now))
        self.table.setItem(row, 1, QTableWidgetItem(os.path.basename(path)))
        self.table.setItem(row, 2, QTableWidgetItem(result))

    def show_next(self):
        if self.current_index < len(self.cycle_images) - 1:
            self.current_index += 1
            self.show_image()

    def show_prev(self):
        if self.current_index > 0:
            self.current_index -= 1
            self.show_image()

    def reset_results(self):
        self.table.setRowCount(0)
        self.progress.setValue(0)
        self.lbl_sim.setText("Similarity: N/A")
        self.lbl_image.setPixmap(QPixmap())
        self.lbl_image.setText("No Image")
        self.current_index = 0
        self.lbl_prev.setEnabled(False)
        self.lbl_next.setEnabled(False)

    def save_to_csv(self):
        path, _ = QFileDialog.getSaveFileName(self, "Save CSV", "", "CSV files (*.csv)")
        if path:
            with open(path, "w", newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Time", "Image Name", "Result"])
                for row in range(self.table.rowCount()):
                    row_data = [self.table.item(row, col).text() for col in range(3)]
                    writer.writerow(row_data)


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = DetectionUI()
    window.show()
    sys.exit(app.exec_())